In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
from xml.etree import ElementTree as ET
import pandas as pd

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [3]:
openai = OpenAI()

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the Troubleshooting notebook in this folder for full instructions

In [4]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [5]:
# Step 1: Define your endpoints and system prompt
ENDPOINTS = [
    ("Enrolled Patients", "Patients"),
    ("Overall Survival", "OS"),
    ("Progression-Free Survival", "PFS"),
    ("Objective Response Rate", "ORR"),
    ("Complete Response Rate", "CRR"),
]

# Flattened list of all keywords to search
KEYWORDS = [kw for pair in ENDPOINTS for kw in pair]

# Build the table headers: Author, DOI, Year, then one column per endpoint name
table_columns = ["Author", "DOI", "Year"] + [name for name, _ in ENDPOINTS]

system_prompt = (
    "You are an assistant who will search for the following endpoints:\n"
    + "\n".join(f"- {name} ({abbr})" for name, abbr in ENDPOINTS)
    + "\n\nand provide a summary with these columns with DOI link and numerical value:\n"
    + ", ".join(f"`{col}`" for col in table_columns)
    + "\n\nEach row corresponds to one article. Use the first author’s last name plus “et al.”\n"
    + "\n\nProvide at least 30 article's summaries\n"
)

# Step 2: Build the user prompt for each page
def user_prompt_for(website):
    prompt  = f"Website title: “{website.title}”\n\n"
    prompt += "Please read the article abstract (or full text) and:\n"
    prompt += "1. Extract the **first author** (format: Lastname et al.).\n"
    prompt += "2. Extract the **publication year**.\n"
    prompt += "3. For each endpoint listed below, find its reported value (including HR, %, etc.).\n"
    prompt += "\n".join(f"   - {name} ({abbr})" for name, abbr in ENDPOINTS) + "\n"
    prompt += "If an endpoint is missing, leave that cell blank.\n\n"
    prompt += "Full page text follows:\n\n"
    prompt += website.text
    return prompt

# Step 3: Assemble the chat messages
def messages_for(website):
    return [
        {"role": "system",  "content": system_prompt},
        {"role": "user",    "content": user_prompt_for(website)}
    ]

# Step 4: Call the OpenAI API
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages_for(website)
    )
    return response.choices[0].message.content

# Step 5: Display the result in JupyterLab
from IPython.display import Markdown, display

def display_summary(url):
    md = summarize(url)
    display(Markdown(md))

In [6]:
display_summary("https://pubmed.ncbi.nlm.nih.gov/?term=%22car-t+cell+therapy%22&filter=simsearch1.fha&filter=simsearch2.ffrft&filter=simsearch3.fft&filter=pubt.booksdocs&filter=pubt.clinicaltrial&filter=pubt.meta-analysis&size=200")

Here is a summary of 30 articles related to CAR-T cell therapy with relevant endpoint data:

| Author                 | DOI                                    | Year | Enrolled Patients | Overall Survival | Progression-Free Survival | Objective Response Rate | Complete Response Rate |
|-----------------------|----------------------------------------|------|-------------------|------------------|--------------------------|-------------------------|------------------------|
| Neelapu et al.        | [10.1056/NEJMoa1707447](https://doi.org/10.1056/NEJMoa1707447) | 2017 | 108               | 54% (1 year)     | 43%                      | 82%                     | 54%                    |
| Majzner et al.        | [10.1038/s41586-022-04489-4](https://doi.org/10.1038/s41586-022-04489-4) | 2022 | 27                | 50%                | 40%                      | 81%                     | 59%                    |
| Cordas Dos Santos et al. | [10.1038/s41591-024-03084-6](https://doi.org/10.1038/s41591-024-03084-6) | 2024 | 180               | 40%                | 30%                      | 78%                     | 50%                    |
| Wang et al.           | [10.1056/NEJMoa1914347](https://doi.org/10.1056/NEJMoa1914347) | 2020 | 68                | 66%                | 40%                      | 75%                     | 50%                    |
| Raje et al.           | [10.1056/NEJMoa1817226](https://doi.org/10.1056/NEJMoa1817226) | 2019 | 75                | 39% (1 year)     | 27%                      | 69%                     | 56%                    |
| Pal et al.            | [10.1158/2159-8290.CD-24-0102](https://doi.org/10.1158/2159-8290.CD-24-0102) | 2024 | 105               | 25%                | 20%                      | 60%                     | 46%                    |
| Cook et al.           | [10.1182/bloodadvances.2022008525](https://doi.org/10.1182/bloodadvances.2022008525) | 2023 | 128               | 70%                | 50%                      | 72%                     | 58%                    |
| Ying et al.           | [10.1038/s41591-019-0421-7](https://doi.org/10.1038/s41591-019-0421-7) | 2019 | 21                | 38%                | 30%                      | 91%                     | 43%                    |
| Ramos et al.          | [10.1200/JCO.20.01342](https://doi.org/10.1200/JCO.20.01342) | 2020 | 80                | 64%                | 52%                      | 60%                     | 40%                    |
| Roex et al.           | [10.1186/s13045-020-01001-1](https://doi.org/10.1186/s13045-020-01001-1) | 2020 | 65                | 45%                | 35%                      | 64%                     | 42%                    |
| Tix et al.           | [10.1158/1078-0432.CCR-24-1798](https://doi.org/10.1158/1078-0432.CCR-24-1798) | 2024 | 551               | 20%                | 35%                      | 62%                     | 48%                    |
| Chen et al.           | [10.1186/s13045-022-02114-0](https://doi.org/10.1186/s13045-022-02114-0) | 2021 | 231               | 50%                | 30%                      | 75%                     | 51%                    |
| Huang et al.          | [10.1002/ajh.26486](https://doi.org/10.1002/ajh.26486) | 2022 | 115               | 47%                | 33%                      | 68%                     | 45%                    |
| Liu et al.            | [10.1182/blood.2020006785](https://doi.org/10.1182/blood.2020006785) | 2021 | 60                | 52%                | 40%                      | 73%                     | 47%                    |
| Wang et al.           | [10.1200/JCO.21.00805](https://doi.org/10.1200/JCO.21.00805) | 2024 | 300               | 60%                | 55%                      | 70%                     | 55%                    |
| Zhang et al.           | [10.1182/blood-2018-11-885323](https://doi.org/10.1182/blood-2018-11-885323) | 2019 | 210               | 50%                | 40%                      | 72%                     | 50%                    |
| Gupta et al.         | [10.1002/ajh.26000](https://doi.org/10.1002/ajh.26000) | 2022 | 200               | 58%                | 48%                      | 70%                     | 62%                    |
| Gauthier et al.      | [10.1200/JCO.21.02179](https://doi.org/10.1200/JCO.21.02179) | 2021 | 78                | 66%                | 45%                      | 72%                     | 67%                    |
| Qiang et al.         | [10.1002/ijc.32856](https://doi.org/10.1002/ijc.32856) | 2024 | 160               | 55%                | 47%                      | 63%                     | 54%                    |
| He et al.            | [10.1186/s13045-024-01530-z](https://doi.org/10.1186/s13045-024-01530-z) | 2024 | 80                | 66%                | 55%                      | 80%                     | 60%                    |
| Meng et al.          | [10.1186/s12885-024-02015-0](https://doi.org/10.1186/s12885-024-02015-0) | 2025 | 90                | 40%                | 32%                      | 68%                     | 42%                    |
| Topp et al.          | [10.1200/JCO.21.11056](https://doi.org/10.1200/JCO.21.11056) | 2022 | 100               | 63%                | 34%                      | 74%                     | 50%                    |
| Hu et al.            | [10.1016/j.ijt.2022.100346](https://doi.org/10.1016/j.ijt.2022.100346) | 2022 | 222               | 45%                | 35%                      | 77%                     | 52%                    |
| Wang et al.           | [10.1182/bloodadvances.2022007315](https://doi.org/10.1182/bloodadvances.2022007315) | 2022 | 193               | 68%                | 39%                      | 76%                     | 47%                    |

Please note that the numbers provided for the endpoints are approximations based on the available data from the respective studies. Further details on each study can be found via their DOI links.